# Artificial Neural Network

# DataPreprocessing 

In [37]:
# import all the libraries
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
import keras
import pickle
from  keras.models import Sequential
from sklearn.metrics import  *
from keras.layers import Dense
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

In [38]:
# import dataset
dataset = pd.read_csv('Data/Churn_Modelling.csv')

In [39]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [40]:
# split dataset into train and test
train,test  = train_test_split(dataset,test_size=0.2,random_state=0)

In [41]:
# saving test data into seperate file
# file = open('Test_data/test.pkl','wb')
# pickle.dump(test,file)
# file.close()
test.to_csv('Test_data/test_file.csv',index=False,encoding='utf-8')

In [42]:
# split train data into train and cross validation 
train_data,cross_val = train_test_split(train,test_size=0.2,random_state=0)

In [43]:
# seperating features and labels from dataset
# Credit Score through Estimated Salary
x_train = train_data.iloc[:,3:13].values
# Exited
y_train = train_data.iloc[:,13].values

In [44]:
x_train.shape,y_train.shape

((6400, 10), (6400,))

The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 < 2). Therefore, one way to get rid of that problem is to split the countries into respective dimensions.

In [45]:
def categorical_encode(x):
    # Encoding categorical data country
    labelencoder_x_country = LabelEncoder()
    x[:,1] = labelencoder_x_country.fit_transform(x[:,1])
    # Encode categorical data gender
    labelencoder_x_gender = LabelEncoder()
    x[:,2]  =  labelencoder_x_gender.fit_transform(x[:,2])
    ohe_country = OneHotEncoder(categorical_features=[1])
    x = ohe_country.fit_transform(x).toarray()
    x = x[:, 1:]
    return x

x_train = categorical_encode(x_train)

/home/admin1/Downloads/Enter/envs/my_env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [46]:
# seperating features and labels from cross val
# Credit Score through Estimated Salary
x_cv = cross_val.iloc[:,3:13].values
# Exited
y_cv = cross_val.iloc[:,13].values

In [47]:
# one hot encoding on cross validation
x_cv = categorical_encode(x_cv)

/home/admin1/Downloads/Enter/envs/my_env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [48]:
# Feature scaling on independent variables
def feature_scaling(x_data):
    sc = StandardScaler()
    x_data = sc.fit_transform(x_data)
    return x_data,sc

In [49]:
# scaling on train data
x_train,sc_train = feature_scaling(x_train)

In [50]:
# scaling on cross val
x_cv,sc_cv = feature_scaling(x_cv)

In [51]:
x_train.shape,y_train.shape,x_cv.shape,y_cv.shape

((6400, 11), (6400,), (1600, 11), (1600,))

# ANN using Keras

In [52]:
# initialize ANN
classifier = Sequential()

The breakdown of the inputs for the first layer is as follows:

activiation:relu
       becasue we are in an input layer. uses the ReLu activation function for  ϕ 
input_dim: 11 because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

units: 6 nodes (number of nodes in hidden layer). Can think of this as number of nodes are in the next layer.

kernel_initializer: uniform the distribution with which we randomly initialize weights for the nodes in this layer.

In [53]:
# added first input layer(by specifying input dimensions) and hidden layer(by units)
classifier.add(Dense(activation='relu',input_dim=11,units=6, kernel_initializer='uniform'))
# We are going to add another layer to this model because we want to implement Deep Learning, 
# which is an artificial Neural network with many layers.
classifier.add(Dense(activation='relu',units=6, kernel_initializer='uniform'))
#  Adding the output layer
# We use the sigmoid because we want probability outcomes whether customer will stay or exit
classifier.add(Dense(activation='sigmoid',units=1,kernel_initializer='uniform'))

The breakdown of the inputs for compiling is as follows:
    
optimizer:
    adam The algorithm we want to use to find the optimal set of weights in the neural networks. 
    Adam is a very efficeint variation of Stochastic Gradient Descent.

loss: binary_crossentropy 
    This is the loss function used within adam. This should be the logarthmic loss. 
    If our dependent (output variable) is Binary, it is binary_crossentropy.
    If Categorical, then it is called categorical_crossentropy

metrics: [accuracy] 
    The accuracy metrics which will be evaluated(minimized) by the model. 
    Used as accuracy criteria to imporve model performance.

kernel_initializer: uniform 
    the distribution with which we randomly initialize weights for the nodes in this layer.

In [54]:
# compiling the neural network
classifier.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [55]:
# fitting the ANN to the training set
classifier.fit(x_train,y_train,batch_size=10,epochs=100)

Epoch 1/100
6400/6400 [==============================] - 1s 109us/step - loss: 0.4987 - acc: 0.7972
Epoch 2/100
6400/6400 [==============================] - 1s 81us/step - loss: 0.4288 - acc: 0.7973
Epoch 3/100
6400/6400 [==============================] - 1s 81us/step - loss: 0.4250 - acc: 0.7973
Epoch 4/100
6400/6400 [==============================] - 1s 82us/step - loss: 0.4205 - acc: 0.8031
Epoch 5/100
6400/6400 [==============================] - 1s 81us/step - loss: 0.4162 - acc: 0.8239
Epoch 6/100
6400/6400 [==============================] - 1s 81us/step - loss: 0.4128 - acc: 0.8278
Epoch 7/100
6400/6400 [==============================] - 0s 77us/step - loss: 0.4102 - acc: 0.8298
Epoch 8/100
6400/6400 [==============================] - 1s 83us/step - loss: 0.4079 - acc: 0.8308
Epoch 9/100
6400/6400 [==============================] - 0s 75us/step - loss: 0.4068 - acc: 0.8322
Epoch 10/100
6400/6400 [==============================] - 1s 78us/step - loss: 0.4046 - acc: 0.8336
Epoch 11

6400/6400 [==============================] - 0s 73us/step - loss: 0.3732 - acc: 0.8384
Epoch 84/100
6400/6400 [==============================] - 0s 73us/step - loss: 0.3727 - acc: 0.8377
Epoch 85/100
6400/6400 [==============================] - 0s 73us/step - loss: 0.3728 - acc: 0.8383
Epoch 86/100
6400/6400 [==============================] - 0s 77us/step - loss: 0.3722 - acc: 0.8409
Epoch 87/100
6400/6400 [==============================] - 0s 72us/step - loss: 0.3732 - acc: 0.8380
Epoch 88/100
6400/6400 [==============================] - 0s 74us/step - loss: 0.3720 - acc: 0.8378
Epoch 89/100
6400/6400 [==============================] - 0s 72us/step - loss: 0.3725 - acc: 0.8405
Epoch 90/100
6400/6400 [==============================] - 0s 76us/step - loss: 0.3716 - acc: 0.8381
Epoch 91/100
6400/6400 [==============================] - 0s 73us/step - loss: 0.3719 - acc: 0.8381
Epoch 92/100
6400/6400 [==============================] - 0s 75us/step - loss: 0.3717 - acc: 0.8392
Epoch 93/100


In [61]:
prediction  = classifier.predict(x_cv)

In [62]:
prediction = (prediction > 0.5)

In [63]:
matrix = confusion_matrix(y_cv,prediction)

In [64]:
print (accuracy_score(y_cv,prediction)*100,'% of cross validation data was classified correctly')

83.9375 % of cross validation data was classified correctly


In [65]:
# save model in pickle file
with open('pickle/save_model.pkl','wb') as f:
    pickle.dump(classifier,f)
    pickle.dump(sc_train,f)